In [2]:
import sklearn
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
import pickle

import os

# Tier 1

In [3]:
full_data = pd.read_csv('../data/data-org/train.csv', sep='\t').set_index('id')

def sigmoid_dim_1(arr):
    e = np.exp(arr[:, 1, :])
    return e / (1+e)

In [34]:
## macbert model

with open('../finetuned_models/ensemble_1/macbert/data/folds.txt', 'r') as f:
    dev_idx_1 = np.array(list(map(int, f.readline().strip().split(' '))))

np.random.seed(57)
tier_1_dev_idx_1 = np.random.choice(range(len(dev_idx_1)), size=len(dev_idx_1) // 5)
tier_1_train_idx_1 = np.array(list(set(range(len(dev_idx_1))) - set(tier_1_dev_idx_1)))

train_1 = full_data.iloc[dev_idx_1[tier_1_train_idx_1]]
dev_1 = full_data.iloc[dev_idx_1[tier_1_dev_idx_1]]

y_train_1 = train_1.label.values
y_dev_1 = dev_1.label.values

preds_part_1 = np.load('../finetuned_models/ensemble_1/macbert/tier0_dev_set_pred_model_0_1.npy')
preds_part_2 = np.load('../finetuned_models/ensemble_1/macbert/tier0_dev_set_pred.npy')
x_1 = sigmoid_dim_1(np.concatenate((preds_part_1, preds_part_2), 2))

x_train_1 = x_1[tier_1_train_idx_1]
x_dev_1 = x_1[tier_1_dev_idx_1]

clf_1 = svm.SVC(
    C=1, 
    gamma="scale", 
    class_weight={0:7, 1:3}, 
    kernel='rbf', 
    random_state=1117, 
    probability=True, 
)
clf_1.fit(x_train_1, y_train_1)

# save the model to disk
dir = '../finetuned_models/ensemble_1/macbert'
filename = 'svm.sav'
pickle.dump(clf_1, open(os.path.join(dir, filename), 'wb'))

pred_dev_1 = clf_1.predict(x_dev_1)

f1_score(y_dev_1, pred_dev_1), accuracy_score(y_dev_1, pred_dev_1)

(0.8633962264150943, 0.7997787610619469)

In [20]:
## roberta model

with open('../finetuned_models/ensemble_1/roberta_word_based/data/folds.txt', 'r') as f:
    dev_idx_2 = np.array(list(map(int, f.readline().strip().split(' '))))

np.random.seed(36)
tier_1_dev_idx_2 = np.random.choice(range(len(dev_idx_2)), size=len(dev_idx_2) // 5)
tier_1_train_idx_2 = np.array(list(set(range(len(dev_idx_2))) - set(tier_1_dev_idx_2)))

train_2 = full_data.iloc[dev_idx_2[tier_1_train_idx_2]]
dev_2 = full_data.iloc[dev_idx_2[tier_1_dev_idx_2]]

y_train_2 = train_2.label.values
y_dev_2 = dev_2.label.values

preds_2 = np.load('../finetuned_models/rww/tier0_dev_set_pred.npy')
x_2 = sigmoid_dim_1(preds_2)

x_train_2 = x_2[tier_1_train_idx_2]
x_dev_2 = x_2[tier_1_dev_idx_2]

clf_2 = svm.SVC(
    C=.1, 
    gamma="scale", 
    # class_weight={0:7, 1:3}, 
    kernel='rbf', 
    random_state=1117, 
    probability=True, 
)
clf_2.fit(x_train_2, y_train_2)

# save the model to disk
dir = '../finetuned_models/rww'
filename = 'svm.sav'
pickle.dump(clf_2, open(os.path.join(dir, filename), 'wb'))

pred_dev_2 = clf_2.predict(x_dev_2)

f1_score(y_dev_2, pred_dev_2), accuracy_score(y_dev_2, pred_dev_2)

(0.8432501599488164, 0.7289823008849557)

# Tier 2

In [24]:
y = np.concatenate((y_dev_1, y_dev_2))

var_1 = np.concatenate((clf_1.decision_function(x_dev_1), clf_1.decision_function(x_dev_2)))
var_2 = np.concatenate((clf_2.decision_function(x_dev_1), clf_2.decision_function(x_dev_2)))
x = np.c_[var_1, var_2]

clf = svm.SVC(
    C=1, 
    gamma="scale", 
    # class_weight={0:7, 1:3}, 
    kernel='rbf', 
    random_state=1117, 
)
clf.fit(x, y)

# save the model to disk
dir = '../finetuned_models/ensemble_1'
filename = 'svm.sav'
pickle.dump(clf_2, open(os.path.join(dir, filename), 'wb'))

pred = clf.predict(x)
f1_score(y, pred), accuracy_score(y, pred)

(0.90117994100295, 0.8517699115044248)

# Pred

In [14]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM

from utils import *
from dataset import *
from preprocess import *
from wrapper import *
from models import *
from pipeline import PipelineGED

torch.cuda.empty_cache()

# device = torch.device("cpu")
device = torch.device("cuda:0")
ntf()

Simbert不能正常使用，除非你安装：bert4keras、tensorflow ，为了安装快捷，没有默认安装.... No module named 'bert4keras'


Building prefix dict from d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\holaj\AppData\Local\Temp\jieba.u842d550ffb06c1e965781a2cb0144564.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.022 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\words.vector.gz] ...


In [15]:
test_df = pd.read_csv('../data/data-org/test.csv', sep='\t', index_col='id')

MODEL_CONFIG = {
    'macbert':{
        'model_name':'hfl/chinese-macbert-base', 
        'model_architecture':'bert_with_clf_head', 
        'max_length':128, 
        'checkpoints':[
            '../finetuned_models/ensemble_1/macbert/model0/checkpoint-1142/pytorch_model.bin', 
            '../finetuned_models/ensemble_1/macbert/model1/checkpoint-2278/pytorch_model.bin', 
            '../finetuned_models/ensemble_1/macbert/model2/checkpoint-1137/pytorch_model.bin', 
            '../finetuned_models/ensemble_1/macbert/model3/checkpoint-1138/pytorch_model.bin', 
            '../finetuned_models/ensemble_1/macbert/model4/checkpoint-1141/pytorch_model.bin', 
        ], 
    }, 
    'roberta-word-based':{
        'model_name':'uer/roberta-base-word-chinese-cluecorpussmall', 
        'model_architecture':'bert_word_based', 
        'max_length':64, 
        'checkpoints':[
            '../finetuned_models/rww/model0/checkpoint-2492/pytorch_model.bin', 
            '../finetuned_models/rww/model1/checkpoint-2478/pytorch_model.bin', 
            '../finetuned_models/rww/model2/checkpoint-2474/pytorch_model.bin', 
            '../finetuned_models/rww/model3/checkpoint-2468/pytorch_model.bin', 
            '../finetuned_models/rww/model4/checkpoint-1238/pytorch_model.bin', 
        ], 
    }, 
}

In [16]:
## useful setup functions
def get_dataset(model_name, oob_model_name, max_length):
    test_dataset_config = {
        'model_name':model_name,
        'aux_model_name':oob_model_name,
        'maxlength':max_length,
        'train_val_split':-1,
        'test':True, 
        'remove_username':False,
        'remove_punctuation':False, 
        'to_simplified':False, 
        'emoji_to_text':False, 
        'device':device,
        'split_words':False, 
        'cut_all':False, 
    }

    test = DatasetWithAuxiliaryEmbeddings(df=test_df.reset_index(), **test_dataset_config)
    test.tokenize()
    test.construct_dataset()
    return test

def load_model_configs(MODEL_ARCH):
    max_length = MODEL_CONFIG[MODEL_ARCH]['max_length']
    model_name = MODEL_CONFIG[MODEL_ARCH]['model_name']
    model_architecture = MODEL_CONFIG[MODEL_ARCH]['model_architecture']
    checkpoints = MODEL_CONFIG[MODEL_ARCH]['checkpoints']
    oob_model_name = None
    return model_name, oob_model_name, model_architecture, max_length, checkpoints

def get_pipeline(model_name, oob_model_name, max_length, model_architecture, checkpoints):
    data_configs = {
        'model_name':model_name,
        'maxlength':max_length,
        'train_val_split':-1,
        'test':True, 
        'remove_username':False,
        'remove_punctuation':False, 
        'to_simplified':False, 
        'emoji_to_text':False, 
        'split_words':False, 
        'cut_all':False, 
    }
    clf = PipelineGED(
        model_name=model_name, 
        oob_model_name=oob_model_name, 
        data_configs=data_configs, 
        model_architecture=model_architecture, 
        hidden_layer_size=max_length, 
    )

    def apply_ged_pipeline_oob(texts, checkpoints=checkpoints, majority_vote=False):
        probs = clf(
            texts=texts, 
            checkpoints=checkpoints, 
            device=device, 
            output_probabilities=True, 
            display=False, 
            aggregate=False, 
        )
        return probs

    return clf, apply_ged_pipeline_oob

### Macbert

In [13]:
MODEL_ARCH = 'macbert'

model_name, oob_model_name, model_architecture, max_length, checkpoints = load_model_configs(MODEL_ARCH)
test = get_dataset(model_name, oob_model_name, max_length)

_, pipe_1 = get_pipeline(model_name, oob_model_name, max_length, model_architecture, checkpoints)

macbert_logits = pipe_1(texts=test_df.text.values, checkpoints=checkpoints, )

macbert_probs = sigmoid_dim_1(macbert_logits.cpu().numpy())
macbert_pred = clf_1.decision_function(macbert_probs)

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 65/65 [00:06<00:00, 10.07it/s]


In [15]:
data = test_df.copy(deep=True)
data['comp'] = pd.read_csv('../data/data-org/labeled_test.csv').set_index('id').label

data['pred'] = (macbert_pred > 0).astype(int)
data = data[['comp', 'pred', 'text']]

print(len(data[data.comp != data.pred]))
full_display(data[data.comp != data.pred])

430


,comp,pred,text
id,,,
1,1,0,唐诗宋词是我国浩如烟海的古代文化中一块瑰丽的瑰宝，所以我们要好好珍惜，将它们传承下去。
4,0,1,外公看到小明非常开心，不由分说地把他拉到了书房。
8,0,1,王小红在这次考试中取得了好成绩，校长亲自为他颁发了奖状。
10,0,1,妹妹总是跟在哥哥的后面，想要哥哥给自己买糖吃。
11,1,0,王磊是个著名的画家，能画出精妙绝伦的佳作。
18,0,1,我国生产的石油，长期不能自给。
22,0,1,厦门的彩虹沙滩很宽很美，小红很羡慕生活在海滩的人，因为他们出门就可以赶海。
24,0,1,每次到姥姥家，她都会做上一桌美味的食物，而我都会吃好多，直到把肚皮撑得溜圆。
25,1,0,电影《流浪地球》一经上映，刘慈欣的同名小说也备受关注，观众认为其气势恢宏、令人震撼。


### RWW

In [17]:
MODEL_ARCH = 'roberta-word-based'

model_name, oob_model_name, model_architecture, max_length, checkpoints = load_model_configs(MODEL_ARCH)
test = get_dataset(model_name, oob_model_name, max_length)

_, pipe_2 = get_pipeline(model_name, oob_model_name, max_length, model_architecture, checkpoints)

rww_logits = pipe_2(texts=test_df.text.values, checkpoints=checkpoints, )

rww_probs = sigmoid_dim_1(rww_logits.cpu().numpy())
rww_pred = clf_2.decision_function(rww_probs)

Some weights of the model checkpoint at uer/roberta-base-word-chinese-cluecorpussmall were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at uer/roberta-base-word-chinese-cluecorpussmall and are newly initialized: ['bert

In [21]:
data = test_df.copy(deep=True)
data['comp'] = pd.read_csv('../data/data-org/labeled_test.csv').set_index('id').label

data['pred'] = (rww_pred > 0).astype(int)
data = data[['comp', 'pred', 'text']]

print(len(data[data.comp != data.pred]))
full_display(data[data.comp != data.pred])

511


,comp,pred,text
id,,,
8,0,1,王小红在这次考试中取得了好成绩，校长亲自为他颁发了奖状。
10,0,1,妹妹总是跟在哥哥的后面，想要哥哥给自己买糖吃。
12,0,1,关于长城，民间一直都有这样的说法“不到长城非好汉”。
13,0,1,官员和民众都必须遵守法律法规，否则就会受到惩罚。
18,0,1,我国生产的石油，长期不能自给。
22,0,1,厦门的彩虹沙滩很宽很美，小红很羡慕生活在海滩的人，因为他们出门就可以赶海。
24,0,1,每次到姥姥家，她都会做上一桌美味的食物，而我都会吃好多，直到把肚皮撑得溜圆。
26,0,1,刘成刚刚来到学校，就有很多老师关注他，因为他在最近的一次考试中取得了第一名的好成绩。
30,0,1,为了避免交通事故的发生，我们应该时刻遵守交通规则。


In [30]:
data.to_csv('../finetuned_models/ensemble_1/roberta_word_based/case-study/trial1.csv', encoding='utf-8')

In [18]:
pred = np.c_[macbert_pred, rww_pred]
final_pred = clf.predict(pred)

In [26]:
data = test_df.copy(deep=True)
data['comp'] = pd.read_csv('../data/data-org/labeled_test.csv').set_index('id').label

data['pred'] = final_pred
data = data[['comp', 'pred', 'text']]

print(len(data[data.comp != data.pred]))
full_display(data[data.comp != data.pred])

365


,comp,pred,text
id,,,
1,1,0,唐诗宋词是我国浩如烟海的古代文化中一块瑰丽的瑰宝，所以我们要好好珍惜，将它们传承下去。
2,1,0,这次迎新活动举办得非常成功，参加活动的人数超过1000人。
3,1,0,春天到了，山上姹紫嫣红的野花开得灿烂极了。
4,0,1,外公看到小明非常开心，不由分说地把他拉到了书房。
8,0,1,王小红在这次考试中取得了好成绩，校长亲自为他颁发了奖状。
10,0,1,妹妹总是跟在哥哥的后面，想要哥哥给自己买糖吃。
11,1,0,王磊是个著名的画家，能画出精妙绝伦的佳作。
18,0,1,我国生产的石油，长期不能自给。
22,0,1,厦门的彩虹沙滩很宽很美，小红很羡慕生活在海滩的人，因为他们出门就可以赶海。


In [25]:
data

,comp,pred,text
id,,,
1,1,0,唐诗宋词是我国浩如烟海的古代文化中一块瑰丽的瑰宝，所以我们要好好珍惜，将它们传承下去。
2,1,0,这次迎新活动举办得非常成功，参加活动的人数超过1000人。
3,1,0,春天到了，山上姹紫嫣红的野花开得灿烂极了。
4,0,1,外公看到小明非常开心，不由分说地把他拉到了书房。
5,1,1,今天的解方程太难了，晓军用了整整一个小时左右才做完。
...,...,...,...
1035,0,0,大年初一，我和父母一起观看了电影《长津湖》，我们深受感动。
1036,0,0,马路的两旁种上了高大的白杨树。
1037,1,1,在出发之前，导游已经三令五申强调不要私自离队，独自游玩。


In [9]:
pred = pd.read_csv('../submissions/submission-v3-0.75624.csv', delimiter='\t').set_index('id').label.values
annotations = pd.read_csv('../data/data-org/labeled_test.csv').set_index('id').label.values

f1_score(annotations, pred)

0.7371134020618556